In [ ]:
import aliases

In [ ]:
from everest import incision

In [ ]:
myobj = incision.Incisable()

In [ ]:
myobj

- A Metric is a one-dimensional Incisable; something analogous to a Pandas Series (as opposed to a Pandas Dataframe).
- A Metric should behave like a Set. A pure metric is simply a system of discrimination between

In [ ]:
class Metric(incision.Incisable):

    ...


class Ordered(Metric):

    ...

In [ ]:
class MyMeta(type):

    def __contains__(cls, arg):
        return bool(arg)


class MyClass(metaclass=MyMeta):
    ...

In [ ]:
1 in MyClass

In [ ]:
class MyClass:
    def mymeth(self, foo):
        return foo
    __call__ = mymeth

In [ ]:
callable(MyClass)